In [222]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
pd.set_option('display.max_columns', 100)

# AQJ User Acquisition Data

In [6]:
finup_aqj_data = pd.read_csv('data/raw/finup_aqj_data.csv')

In [164]:
finup_aqj_data.shape

(61149, 56)

In [105]:
finup_aqj_data.head(20)

,user_id,mobile,isinvested,user_age,sex_id,create_date_id,create_time,date_id,invest_time,areacode,plus_date,client_type_id,channel_id,pno,isrecharged,rechargestatus,isinvited,lastday_invite,lastamt_invite,callstatus,call_effec,gap_calltoinvest,last_calltime,call_times,staytime,staynum,staytime_recharge,staynum_recharge,staytime_p3,staynum_p3,staytime_p6,staynum_p6,staytime_p12,staynum_p12,staytime_ph,staynum_ph,staytime_p1,staynum_p1,staytime_buy,staynum_buy,staytime_asset,staynum_asset,staytime_redbag,staynum_redbag,staytime_hudong,staynum_hudong,staynum_banner,staynum_icon1,staynum_icon2,staynum_icon3,devicetype,isactive_aftercall,active_num,lastday_before_invest,lianxuday_last,rate
0,11427132,xy6b0db75e28da240d2aba3c4621c78ca020160926,已投资,33.0,2,20170921,2017-09-21 20:32:51.0,20170921.0,2017-09-21 20:53:32.0,620100.0,0.0,3,1438,863996035381438,1,1.0,0,NaN,NaN,0,失败,NaN,NaN,NaN,729.0,52,163.0,5,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,9.0,1,24.0,4,NaN,0,NaN,0,0,0,0,0,vivo Y66,0,1.0,NaN,0.0,0.137931
1,11384619,xy1d8cf081de5cb9ea1b8e6722376870c120160926,未投资,NaN,0,20170921,2017-09-21 09:05:24.0,NaN,NaN,430100.0,NaN,3,1438,A10000554C9232,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,272.0,31,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,15.0,3,0,0,0,0,vivo V3M A,0,1.0,NaN,0.0,0.137931
2,11420933,xyd0787171f3c126d33c17788d11ded9fa20160926,未投资,28.0,1,20170921,2017-09-21 18:31:19.0,NaN,NaN,520400.0,NaN,3,1438,862732037361891,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,416.0,27,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,15.0,3,NaN,0,NaN,0,NaN,0,NaN,0,0,0,0,0,OPPO R9 Plusm A,0,1.0,NaN,0.0,0.137931
3,11383761,xyf6e4d4be5264e16e004ad0c11516a54020160926,未投资,NaN,0,20170921,2017-09-21 08:46:03.0,NaN,NaN,371700.0,NaN,3,1438,861010032382506,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,139.0,19,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0,0,0,0,HUAWEI MLA-AL00,0,1.0,NaN,0.0,0.137931
4,11392954,xy0908a1e652214f2541e083d0cc22da0620160926,未投资,46.0,1,20170921,2017-09-21 11:10:49.0,NaN,NaN,422800.0,NaN,3,1438,865578034261379,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,363.0,27,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,10.0,1,NaN,0,54.0,3,0,0,0,0,HUAWEI MLA-AL10,0,1.0,NaN,0.0,0.137931
5,11438962,xyfe7a0c706e27e591b6f483fdc885791d20160926,未投资,25.0,2,20170921,2017-09-21 23:52:44.0,NaN,NaN,341300.0,NaN,3,1438,A100005A1C1626,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,390.0,52,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,48.0,7,15.0,1,23.0,7,0,0,1,0,vivo Y66,0,1.0,NaN,0.0,0.137931
6,11413974,xy47ac142162e9c17cfdef3f87b1f28b6d20160926,未投资,NaN,0,20170921,2017-09-21 16:13:47.0,NaN,NaN,130200.0,NaN,3,1438,351702076704176,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,435.0,33,NaN,0,NaN,0,NaN,0,NaN,0,13.0,2,17.0,3,10.0,1,NaN,0,NaN,0,NaN,0,0,0,0,0,SM-J7008,0,1.0,NaN,0.0,0.137931
7,11397486,xy34ed72dea8537624d941b4d0b5f27b5320160926,未投资,NaN,0,20170921,2017-09-21 12:14:18.0,NaN,NaN,429004.0,NaN,3,1438,869515023973560,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,694.0,58,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,36.0,1,NaN,0,NaN,0,22.0,2,0,3,2,0,m1 metal,0,1.0,NaN,0.0,0.137931
8,11434283,xy749bae0dd107de41c60c00c3730bea8620160926,未投资,51.0,2,20170921,2017-09-21 22:28:07.0,NaN,NaN,650100.0,NaN,3,1438,a0000055c966c9,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,502.0,40,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,33.0,1,NaN,0,26.0,3,23.0,3,11.0,2,0,0,0,0,HUAWEI GRA-CL10,0,1.0,NaN,0.0,0.137931
9,11379160,xy9ef388548e3499f19809451d88d2699c20160926,未投资,24.0,1,20170921,2017-09-21 04:32:53.0,NaN,NaN,621000.0,NaN,3,1438,35391007036017,0,NaN,0,NaN,NaN,0,失败,NaN,NaN,NaN,372.0,36,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,9.0,3,3.0,1,NaN,0,0,1,0,0,SM-W2016,0,1.0,NaN,0.0,0.137931


In [221]:
len(finup_aqj_data['gap_calltoinvest'].value_counts())

4

In [226]:
def create_dataset_from_data_column(iterable, label, vector_dim=500, num_rows=1000):        
    iterable = np.array(iterable)    
    choice_range = len(iterable)
    iterable_str = iterable.astype(str)
    
    def contains_time_characters(string):
        count = 0
        time_chars = {':', '/', '-', '_', '.', ',', '\\',
                     'hr', 'hour', 'min', 'minute', 'sec', 'second', 'day', 'week', 'year', 
                     '时', '分', '秒', '周', '日', '月', '年', 
                     '時', '分', '秒', '週', '日', '月', '年'}
        for char in time_chars:
            if char in string:
                count += 1
        return count
    
    def is_nan(string):
        return 1 if string.lower() == 'nan' else 0
    
    vector_list = []
    for i in tqdm(list(range(num_rows))):
        indices = np.random.choice(choice_range, vector_dim)
        stringified_data = iterable_str[indices]
        
#         raw_feature_names = ['length', 'range', 'sum', 'avg', 'std', 'cv', 'float', 'time']
        raw_feature_names = ['length', 'range', 'sum', 'avg', 'std', 'float', 'time', 'nan']
        
        raw_feature_dict = {
            'length': np.vectorize(len)(stringified_data),
            'range': np.vectorize(lambda x: max([ord(char) for char in x]) - min([ord(char) for char in x]))(stringified_data),
            'sum': np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data),
            'avg': np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data) / np.vectorize(len)(stringified_data),
            'std': np.vectorize(lambda x: np.array([ord(char) for char in x]).std())(stringified_data),
#             'cv': (np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data) / np.vectorize(len)(stringified_data)) / np.vectorize(lambda x: np.array([ord(char) for char in x]).std())(stringified_data),
            'float': np.vectorize(lambda x: x.count('.'))(stringified_data),
            'time': np.vectorize(contains_time_characters)(stringified_data),
            'nan': np.vectorize(is_nan)(stringified_data)
        }
#         length_data = np.vectorize(len)(stringified_data)
#         range_data = np.vectorize(lambda x: max([ord(char) for char in x]) - min([ord(char) for char in x]))(stringified_data)
#         sum_data = np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data)
#         avg_data = sum_data / length_data
#         std_data = np.vectorize(lambda x: np.array([ord(char) for char in x]).std())(stringified_data)
#         cv_data = avg_data / std_data
#         float_data = np.vectorize(lambda x: x.count('.'))(stringified_data)
#         time_data = np.vectorize(contains_time_characters)(stringified_data)
        
        range_feature_dict = {feature_name + '_range': 
           np.array([raw_feature_dict[feature_name].max() - raw_feature_dict[feature_name].min()]) for feature_name in raw_feature_names
        }
        
#         length_data_range = np.array([length_data.max() - length_data.min()])
#         range_data_range = np.array([range_data.max() - range_data.min()])
#         sum_data_range = np.array([sum_data.max() - sum_data.min()])
        
        sum_feature_dict = {feature_name + '_sum': 
           np.array([raw_feature_dict[feature_name].sum()]) for feature_name in raw_feature_names
        }
        
        avg_feature_dict = {feature_name + '_avg': 
           np.array([raw_feature_dict[feature_name].mean()]) for feature_name in raw_feature_names
        }
                     
        std_feature_dict = {feature_name + '_std': 
           np.array([raw_feature_dict[feature_name].std()]) for feature_name in raw_feature_names
        }
        
        count_distinct_feature_dict = {feature_name + '_distinct': 
           np.array([len(Counter(raw_feature_dict[feature_name]))]) for feature_name in raw_feature_names
        }
        
#         cv_feature_dict = {feature_name + '_cv': 
#            np.array([raw_feature_dict[feature_name].std() / raw_feature_dict[feature_name].mean()]) for feature_name in raw_feature_names
#         }
        
        concat_list = [value for key, value in raw_feature_dict.items()] \
        + [value for key, value in sum_feature_dict.items()] \
        + [value for key, value in avg_feature_dict.items()] \
        + [value for key, value in std_feature_dict.items()] \
        + [value for key, value in count_distinct_feature_dict.items()]
#         + [value for key, value in cv_feature_dict.items()]
        
        vec = np.concatenate(concat_list)
        
#         vec = np.concatenate((length_data, sum_data, avg_data, std_data, float_data, time_data))
        vector_list.append(vec)
        
    return np.array(vector_list), np.array([label] * num_rows)

In [ ]:
aqj_interm_data = {
    'cat1': create_dataset_from_data_column(finup_aqj_data['user_id'], 2),
    'cat2': create_dataset_from_data_column(finup_aqj_data['mobile'], 2),
    'cat3': create_dataset_from_data_column(finup_aqj_data['isinvested'], 2),
    'cat4': create_dataset_from_data_column(finup_aqj_data['sex_id'], 2),
    'cat5': create_dataset_from_data_column(finup_aqj_data['areacode'], 2),
    'cat6': create_dataset_from_data_column(finup_aqj_data['client_type_id'], 2),
    'cat7': create_dataset_from_data_column(finup_aqj_data['channel_id'], 2),
    'cat8': create_dataset_from_data_column(finup_aqj_data['pno'], 2),
    'cat9': create_dataset_from_data_column(finup_aqj_data['isrecharged'], 2),
    'cat10': create_dataset_from_data_column(finup_aqj_data['rechargestatus'], 2),
    'cat11': create_dataset_from_data_column(finup_aqj_data['call_effec'], 2),
    'num1': create_dataset_from_data_column(finup_aqj_data['user_age'], 0),
    'num2': create_dataset_from_data_column(finup_aqj_data['plus_date'], 0),
    'num3': create_dataset_from_data_column(finup_aqj_data['staytime'], 0),
    'num4': create_dataset_from_data_column(finup_aqj_data['staynum'], 0),
    'num5': create_dataset_from_data_column(finup_aqj_data['staytime_recharge'], 0),
    'num6': create_dataset_from_data_column(finup_aqj_data['rate'], 0),
    'num7': create_dataset_from_data_column(finup_aqj_data['staytime_p12'], 0),
    'num8': create_dataset_from_data_column(finup_aqj_data['staytime_p6'], 0),
    'num9': create_dataset_from_data_column(finup_aqj_data['lastamt_invite'], 0),
    'num10': create_dataset_from_data_column(finup_aqj_data['active_num'], 0),
    's_cat1': create_dataset_from_data_column(finup_aqj_data['devicetype'], 1),
    'time1': create_dataset_from_data_column(finup_aqj_data['create_date_id'], 3),
    'time2': create_dataset_from_data_column(finup_aqj_data['create_time'], 3),
    'time3': create_dataset_from_data_column(finup_aqj_data['date_id'], 3),
    'time4': create_dataset_from_data_column(finup_aqj_data['lastday_before_invest'], 3)
}
# cat1 = create_dataset_from_data_column(finup_aqj_data['user_id'], 2)
# cat2 = create_dataset_from_data_column(finup_aqj_data['mobile'], 2)
# cat3 = create_dataset_from_data_column(finup_aqj_data['isinvested'], 2)
# cat4 = create_dataset_from_data_column(finup_aqj_data['sex_id'], 2)
# cat5 = create_dataset_from_data_column(finup_aqj_data['areacode'], 2)
# cat6 = create_dataset_from_data_column(finup_aqj_data['client_type_id'], 2)
# cat7 = create_dataset_from_data_column(finup_aqj_data['channel_id'], 2)
# cat8 = create_dataset_from_data_column(finup_aqj_data['pno'], 2)
# cat9 = create_dataset_from_data_column(finup_aqj_data['isrecharged'], 2)
# cat10 = create_dataset_from_data_column(finup_aqj_data['rechargestatus'], 2)
# cat11 = create_dataset_from_data_column(finup_aqj_data['call_effec'], 2)

# num1 = create_dataset_from_data_column(finup_aqj_data['user_age'], 0)
# num2 = create_dataset_from_data_column(finup_aqj_data['plus_date'], 0)
# num3 = create_dataset_from_data_column(finup_aqj_data['staytime'], 0)
# num4 = create_dataset_from_data_column(finup_aqj_data['staynum'], 0)
# num5 = create_dataset_from_data_column(finup_aqj_data['staytime_recharge'], 0)
# num6 = create_dataset_from_data_column(finup_aqj_data['rate'], 0)

# s_cat1 = create_dataset_from_data_column(finup_aqj_data['devicetype'], 1)

# time1 = create_dataset_from_data_column(finup_aqj_data['create_date_id'], 3)
# time2 = create_dataset_from_data_column(finup_aqj_data['create_time'], 3)
# time3 = create_dataset_from_data_column(finup_aqj_data['date_id'], 3)

 19%|█▉        | 190/1000 [00:06<00:29, 27.61it/s]

In [200]:
len(aqj_interm_data)

26

In [201]:
# features, label = np.concatenate((data1[0], data2[0], data3[0], data4[0], data5[0], data6[0], data7[0], data8[0], data9[0]), axis=0), np.concatenate((data1[1], data2[1], data3[1], data4[1], data5[1], data6[1], data7[1], data8[1], data9[1]))
features, label = np.concatenate([value[0] for key, value in aqj_interm_data.items()], axis=0), np.concatenate([value[1] for key, value in aqj_interm_data.items()])
features, label = pd.DataFrame(features), pd.DataFrame(label)

In [202]:
features.shape, label.shape

((26000, 8024), (26000, 1))

In [203]:
features = features.replace([np.inf, -np.inf], np.nan)
features.fillna({colname: features[colname].mean() for colname in features.columns.values}, inplace=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,7974,7975,7976,7977,7978,7979,7980,7981,7982,7983,7984,7985,7986,7987,7988,7989,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999,8000,8001,8002,8003,8004,8005,8006,8007,8008,8009,8010,8011,8012,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022,8023
0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7374.0,412085.0,51510.625000,2621.301138,0.0,0.0,0.0,8.000,7.374,412.085,51.510625,2.621301,0.000,0.000,0.000,0.000000,1.325188,6.542001,0.817750,0.514978,0.000000,0.000000,0.000000
1,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7306.0,411894.0,51486.750000,2592.194133,0.0,0.0,0.0,8.000,7.306,411.894,51.486750,2.592194,0.000,0.000,0.000,0.000000,1.390814,6.549104,0.818638,0.534963,0.000000,0.000000,0.000000
2,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7345.0,412042.0,51505.250000,2613.228901,0.0,0.0,0.0,8.000,7.345,412.042,51.505250,2.613229,0.000,0.000,0.000,0.000000,1.362342,6.437409,0.804676,0.537358,0.000000,0.000000,0.000000
3,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7376.0,412179.0,51522.375000,2623.957355,0.0,0.0,0.0,8.000,7.376,412.179,51.522375,2.623957,0.000,0.000,0.000,0.000000,1.329144,6.569091,0.821136,0.528046,0.000000,0.000000,0.000000
4,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7352.0,411803.0,51475.375000,2601.045789,0.0,0.0,0.0,8.000,7.352,411.803,51.475375,2.601046,0.000,0.000,0.000,0.000000,1.367515,6.423410,0.802926,0.528927,0.000000,0.000000,0.000000
5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7408.0,411623.0,51452.875000,2632.145296,0.0,0.0,0.0,8.000,7.408,411.623,51.452875,2.632145,0.000,0.000,0.000,0.000000,1.307492,6.541932,0.817741,0.507089,0.000000,0.000000,0.000000
6,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,7357.0,411868.0,51483.500000,2605.383289,0.0,0.0,0.0,8.000,7.357,411.868,51.483500,2.605383,0.000,0.000,0.000,0.000000,1.361452,6.414871,0.801859,0.515003,0.000000,0.000000,0.000000
7,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.

In [204]:
import pickle

features.to_csv('data/interm/features.csv')
label.to_csv('data/interm/label.csv')

In [205]:
features.isnull().sum().sum(), label.isnull().sum().sum()

(0, 0)

In [206]:
np.isinf(features.values).sum()

0

In [207]:
features.shape, label.shape

((26000, 8024), (26000, 1))

In [208]:
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=1113)

ova_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=512, n_jobs=-1, random_state=1113), n_jobs=-1)
ova_clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=512, n_jobs=-1,
            oob_score=False, random_state=1113, verbose=0,
            warm_start=False),
          n_jobs=-1)

In [209]:
ova_clf.score(X_train, y_train), ova_clf.score(X_test, y_test) 

(1.0, 0.99974358974358979)

In [210]:
y_pred = ova_clf.predict(X_test)

In [211]:
y_test[:20]

,0
13029,0
5262,2
9985,2
22345,3
7777,2
12328,0
4944,2
8912,2
12723,0
1621,2


In [212]:
y_pred[:20]

array([0, 2, 2, 3, 2, 0, 2, 2, 0, 2, 3, 0, 2, 2, 2, 2, 0, 2, 0, 3])

In [213]:
len(y_test) * (1 - ova_clf.score(X_test, y_test))

1.9999999999996465

In [214]:
from collections import Counter

def get_data_column_type(iterable, estimator, robustness=0.1, vector_dim=200):        
    iterable = np.array(iterable)
    choice_range = len(iterable)
    iterable_str = iterable.astype(str)
        
    def contains_time_characters(string):
        time_chars = {':', '/', 
                     'hr', 'hour', 'min', 'minute', 'sec', 'second',
                     'day', 'week', 'year'}
        for char in time_chars:
            if char in string:
                return 1
        return 0
    
    def is_nan(string):
        return 1 if string.lower() == 'nan' else 0
    
    vector_list = []
    
    for i in (range(int(100 * robustness))):
        indices = np.random.choice(choice_range, vector_dim)
        stringified_data = iterable_str[indices]
        
        raw_feature_names = ['length', 'range', 'sum', 'avg', 'std', 'float', 'time', 'nan']
        
        raw_feature_dict = {
            'length': np.vectorize(len)(stringified_data),
            'range': np.vectorize(lambda x: max([ord(char) for char in x]) - min([ord(char) for char in x]))(stringified_data),
            'sum': np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data),
            'avg': np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data) / np.vectorize(len)(stringified_data),
            'std': np.vectorize(lambda x: np.array([ord(char) for char in x]).std())(stringified_data),
#             'cv': (np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data) / np.vectorize(len)(stringified_data)) / np.vectorize(lambda x: np.array([ord(char) for char in x]).std())(stringified_data),
            'float': np.vectorize(lambda x: x.count('.'))(stringified_data),
            'time': np.vectorize(contains_time_characters)(stringified_data),
            'nan': np.vectorize(is_nan)(stringified_data)
        }
#         length_data = np.vectorize(len)(stringified_data)
#         range_data = np.vectorize(lambda x: max([ord(char) for char in x]) - min([ord(char) for char in x]))(stringified_data)
#         sum_data = np.vectorize(lambda x: sum([ord(char) for char in x]))(stringified_data)
#         avg_data = sum_data / length_data
#         std_data = np.vectorize(lambda x: np.array([ord(char) for char in x]).std())(stringified_data)
#         cv_data = avg_data / std_data
#         float_data = np.vectorize(lambda x: x.count('.'))(stringified_data)
#         time_data = np.vectorize(contains_time_characters)(stringified_data)
        
        range_feature_dict = {feature_name + '_range': 
           np.array([raw_feature_dict[feature_name].max() - raw_feature_dict[feature_name].min()]) for feature_name in raw_feature_names
        }
        
#         length_data_range = np.array([length_data.max() - length_data.min()])
#         range_data_range = np.array([range_data.max() - range_data.min()])
#         sum_data_range = np.array([sum_data.max() - sum_data.min()])
        
        sum_feature_dict = {feature_name + '_sum': 
           np.array([raw_feature_dict[feature_name].sum()]) for feature_name in raw_feature_names
        }
        
        avg_feature_dict = {feature_name + '_avg': 
           np.array([raw_feature_dict[feature_name].mean()]) for feature_name in raw_feature_names
        }
                     
        std_feature_dict = {feature_name + '_std': 
           np.array([raw_feature_dict[feature_name].std()]) for feature_name in raw_feature_names
        }
        
        count_distinct_feature_dict = {feature_name + '_distinct': 
           np.array([len(raw_feature_dict[feature_name].value_counts())]) for feature_name in raw_feature_names
        }
        
#         cv_feature_dict = {feature_name + '_cv': 
#            np.array([raw_feature_dict[feature_name].std() / raw_feature_dict[feature_name].mean()]) for feature_name in raw_feature_names
#         }
        
        concat_list = [value for key, value in raw_feature_dict.items()] \
        + [value for key, value in sum_feature_dict.items()] \
        + [value for key, value in avg_feature_dict.items()] \
        + [value for key, value in std_feature_dict.items()] \
        + [value for key, value in count_distinct_feature_dict.items()]
        
        vec = np.concatenate(concat_list)
        
        vector_list.append(vec)
    
    prediction = estimator.predict(np.array(vector_list))
    prediction_count = Counter(np.vectorize(lambda x: round(x, 1))(prediction))
    confidence = prediction_count.most_common(1)[0][1] / len(prediction)
#     print(prediction_count.most_common(1)[0][1], len(prediction))
    decode_dict = {0: 'numeric', 1: 'semantic categorical', 2: 'categorical', 3: 'time'}
        
    return decode_dict[round(prediction.mean())], confidence

In [218]:
def get_data_column_type_df(data, estimator, robustness=0.1, vector_dim=200):
    result_dict = {}
    
    if isinstance(data, pd.DataFrame):
        column_names = data.columns.values
        
        for i, colname in tqdm(list(enumerate(column_names))):
            datatype, confidence = get_data_column_type(data[colname], estimator, robustness=robustness, vector_dim=vector_dim)
            result_dict[colname] = datatype, confidence
    else:
        column_names = list(range(data.shape[1]))
        
        for i, colname in tqdm(list(enumerate(column_names))):
            datatype, confidence = get_data_column_type(data[:, colname], estimator, robustness=robustness, vector_dim=vector_dim)
            result_dict[colname] = datatype, confidence
    
    return result_dict

In [219]:
get_data_column_type_df(finup_aqj_data, ova_clf, robustness=0.3, vector_dim=1000)

100%|██████████| 56/56 [01:32<00:00,  1.73s/it]


{'active_num': ('numeric', 1.0),
 'areacode': ('categorical', 1.0),
 'call_effec': ('categorical', 1.0),
 'call_times': ('numeric', 1.0),
 'callstatus': ('categorical', 1.0),
 'channel_id': ('categorical', 1.0),
 'client_type_id': ('categorical', 1.0),
 'create_date_id': ('time', 1.0),
 'create_time': ('time', 1.0),
 'date_id': ('time', 1.0),
 'devicetype': ('semantic categorical', 1.0),
 'gap_calltoinvest': ('numeric', 1.0),
 'invest_time': ('time', 1.0),
 'isactive_aftercall': ('categorical', 1.0),
 'isinvested': ('categorical', 1.0),
 'isinvited': ('categorical', 1.0),
 'isrecharged': ('categorical', 1.0),
 'last_calltime': ('time', 1.0),
 'lastamt_invite': ('numeric', 1.0),
 'lastday_before_invest': ('time', 1.0),
 'lastday_invite': ('numeric', 1.0),
 'lianxuday_last': ('numeric', 1.0),
 'mobile': ('categorical', 1.0),
 'plus_date': ('numeric', 1.0),
 'pno': ('categorical', 1.0),
 'rate': ('numeric', 1.0),
 'rechargestatus': ('categorical', 0.9666666666666667),
 'sex_id': ('categor

In [65]:
wine_raw = pd.DataFrame.from_csv('data_raw/winemag-data_first150k.csv')

In [66]:
wine_raw.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [67]:
get_data_column_type_df(wine_raw, ova_clf)

100%|██████████| 10/10 [00:04<00:00,  2.23it/s]


{'country': ('semantic categorical', 1.0),
 'description': ('semantic categorical', 1.0),
 'designation': ('semantic categorical', 1.0),
 'points': ('categorical', 1.0),
 'price': ('numeric', 1.0),
 'province': ('semantic categorical', 1.0),
 'region_1': ('semantic categorical', 1.0),
 'region_2': ('semantic categorical', 1.0),
 'variety': ('semantic categorical', 1.0),
 'winery': ('semantic categorical', 1.0)}

In [ ]:
wine_raw['points'].value_counts()

In [16]:
wine_points = create_dataset_from_data_column(wine_raw['points'], 0)

100%|██████████| 5000/5000 [02:43<00:00, 50.32it/s] 


In [17]:
features, label = np.concatenate((features, wine_points[0]), axis=0), np.concatenate((label, wine_points[1])) 

In [18]:
features.shape, label.shape

((50000, 400), (50000,))

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=1113)

ova_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=128, n_jobs=-1, random_state=1113))
ova_clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=-1,
            oob_score=False, random_state=1113, verbose=0,
            warm_start=False),
          n_jobs=1)

In [20]:
y_pred = ova_clf.predict(X_test)
ova_clf.score(X_train, y_train), ova_clf.score(X_test, y_test) 

(1.0, 0.99973333333333336)

In [21]:
get_data_column_type_df(wine_raw, ova_clf)

100%|██████████| 10/10 [00:33<00:00,  3.26s/it]


{'country': ('semantic categorical', 1.0),
 'description': ('semantic categorical', 1.0),
 'designation': ('semantic categorical', 1.0),
 'points': ('numeric', 1.0),
 'price': ('numeric', 1.0),
 'province': ('semantic categorical', 1.0),
 'region_1': ('semantic categorical', 1.0),
 'region_2': ('semantic categorical', 1.0),
 'variety': ('semantic categorical', 1.0),
 'winery': ('semantic categorical', 1.0)}

In [22]:
ted_raw = pd.DataFrame.from_csv('data_raw/ted_main.csv')

In [23]:
ted_raw.head()

,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
comments,,,,,,,,,,,,,,,,
4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [41]:
get_data_column_type_df(ted_raw, ova_clf)

100%|██████████| 16/16 [00:09<00:00,  1.77it/s]


{'description': ('semantic categorical', 1.0),
 'duration': ('numeric', 1.0),
 'event': ('semantic categorical', 1.0),
 'film_date': ('time', 1.0),
 'languages': ('categorical', 1.0),
 'main_speaker': ('semantic categorical', 1.0),
 'name': ('semantic categorical', 1.0),
 'num_speaker': ('numeric', 1.0),
 'published_date': ('time', 1.0),
 'ratings': ('semantic categorical', 1.0),
 'related_talks': ('semantic categorical', 1.0),
 'speaker_occupation': ('semantic categorical', 1.0),
 'tags': ('semantic categorical', 1.0),
 'title': ('semantic categorical', 1.0),
 'url': ('semantic categorical', 1.0),
 'views': ('numeric', 1.0)}

In [ ]:
ted_raw['num_speaker'].value_counts()

In [26]:
film_date = create_dataset_from_data_column(ted_raw['film_date'], 3)
languages = create_dataset_from_data_column(ted_raw['languages'], 2)
num_speaker = create_dataset_from_data_column(ted_raw['num_speaker'], 0)
published_date = create_dataset_from_data_column(ted_raw['published_date'], 3)
views = create_dataset_from_data_column(ted_raw['views'], 0)

100%|██████████| 5000/5000 [00:53<00:00, 93.71it/s] 


In [27]:
features, label = np.concatenate((features, film_date[0], languages[0], num_speaker[0], published_date[0], views[0]), axis=0), np.concatenate((label, film_date[1], languages[1], num_speaker[1], published_date[1], views[1]))

In [28]:
features.shape, label.shape

((75000, 400), (75000,))

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=1113)

ova_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=128, n_jobs=-1, random_state=1113))
ova_clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=-1,
            oob_score=False, random_state=1113, verbose=0,
            warm_start=False),
          n_jobs=1)

In [30]:
y_pred = ova_clf.predict(X_test)
ova_clf.score(X_train, y_train), ova_clf.score(X_test, y_test)

(1.0, 0.99986666666666668)

In [31]:
get_data_column_type_df(ted_raw, ova_clf)

100%|██████████| 16/16 [01:34<00:00,  6.03s/it]


{'description': ('semantic categorical', 1.0),
 'duration': ('numeric', 1.0),
 'event': ('semantic categorical', 1.0),
 'film_date': ('time', 1.0),
 'languages': ('categorical', 1.0),
 'main_speaker': ('semantic categorical', 1.0),
 'name': ('semantic categorical', 1.0),
 'num_speaker': ('numeric', 1.0),
 'published_date': ('time', 1.0),
 'ratings': ('semantic categorical', 1.0),
 'related_talks': ('semantic categorical', 1.0),
 'speaker_occupation': ('semantic categorical', 1.0),
 'tags': ('semantic categorical', 1.0),
 'title': ('semantic categorical', 1.0),
 'url': ('semantic categorical', 1.0),
 'views': ('numeric', 1.0)}

In [33]:
ufo_raw = pd.read_csv('data_raw/ufo-sightings.csv', error_bad_lines=False)

b'Skipping line 878: expected 11 fields, saw 12\nSkipping line 1713: expected 11 fields, saw 12\nSkipping line 1815: expected 11 fields, saw 12\nSkipping line 2858: expected 11 fields, saw 12\nSkipping line 3734: expected 11 fields, saw 12\nSkipping line 4756: expected 11 fields, saw 12\nSkipping line 5389: expected 11 fields, saw 12\nSkipping line 5423: expected 11 fields, saw 12\nSkipping line 5614: expected 11 fields, saw 12\nSkipping line 5849: expected 11 fields, saw 12\nSkipping line 6093: expected 11 fields, saw 12\nSkipping line 7516: expected 11 fields, saw 12\nSkipping line 7626: expected 11 fields, saw 12\nSkipping line 8893: expected 11 fields, saw 12\nSkipping line 9015: expected 11 fields, saw 12\nSkipping line 9571: expected 11 fields, saw 12\nSkipping line 9620: expected 11 fields, saw 12\nSkipping line 9751: expected 11 fields, saw 12\nSkipping line 10157: expected 11 fields, saw 12\nSkipping line 10427: expected 11 fields, saw 12\nSkipping line 12035: expected 11 fiel

In [36]:
ufo_raw.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [42]:
get_data_column_type_df(ufo_raw, ova_clf)

100%|██████████| 11/11 [00:06<00:00,  1.63it/s]


{'city': ('semantic categorical', 1.0),
 'comments': ('semantic categorical', 1.0),
 'country': ('numeric', 1.0),
 'date posted': ('time', 1.0),
 'datetime': ('time', 0.8),
 'duration (hours/min)': ('semantic categorical', 1.0),
 'duration (seconds)': ('categorical', 1.0),
 'latitude': ('time', 1.0),
 'longitude': ('time', 1.0),
 'shape': ('semantic categorical', 1.0),
 'state': ('semantic categorical', 1.0)}